<a href="https://colab.research.google.com/github/Foulach/project_an/blob/main/regression_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Première régression linéaire** : nous retirons l'année, la région et le pays du jeu de données.

Le score de bonheur est notre *y* (variable dépendante).
Les variables *x* (indépendantes) sont :

*   PIB par habitant
*   Liberté de faire des choix de vie
*   Générosité
*   Espérance de vie en bonne santé
*   Perceptions de la corruption
*   Soutien social

Nous conservons les données pour toutes les années (2015 - 2023) sans procéder à aucune agrégation. Ainsi, nous avons une ligne pour chaque combinaison d'année, de pays et d'indicateur.


## Data Import

In [ ]:
# import the dataset from google drive

import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

df = pd.read_csv('/content/drive/My Drive/happiness_project/regressions/copie_merged_whr.csv')
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,year,country,region,happiness_score,gdp_per_capita,social_support,healthy_life_expectancy,freedom_to_make_life_choices,generosity,perceptions_of_corruption
0,2015,switzerland,western europe,7.587,1.39651,1.34951,0.94143,0.66557,0.29678,0.41978
1,2015,iceland,western europe,7.561,1.30232,1.40223,0.94784,0.62877,0.43630,0.14145
2,2015,denmark,western europe,7.527,1.32548,1.36058,0.87464,0.64938,0.34139,0.48357
3,2015,norway,western europe,7.522,1.45900,1.33095,0.88521,0.66973,0.34699,0.36503
4,2015,canada,north america and anz,7.427,1.32629,1.32261,0.90563,0.63297,0.45811,0.32957


## Data Exploration and Cleaning


### Supprimer les doublons

In [ ]:
# Check if there are doublons in the df
num_duplicates = df.duplicated().sum()
print(f"Nombre de lignes dupliquées trouvées : {num_duplicates}")

Nombre de lignes dupliquées trouvées : 0


### Traiter les valeurs manquantes

In [ ]:
# Check if there are missing values in the df
missing_values = df.isnull().sum()
print(missing_values)

year                            0
country                         0
region                          0
happiness_score                 0
gdp_per_capita                  0
social_support                  0
healthy_life_expectancy         0
freedom_to_make_life_choices    0
generosity                      0
perceptions_of_corruption       1
dtype: int64


In [ ]:
# Find the location of the  missing value in perceptions_of_corruption column. Return the name of the line
missing_index = df[df['perceptions_of_corruption'].isnull()].index.tolist()
missing_row_name = df.iloc[missing_index]['country'].values[0]
print(f"Le nom de la ligne avec la valeur manquante dans la colonne 'perceptions_of_corruption' est : {missing_row_name}")


Le nom de la ligne avec la valeur manquante dans la colonne 'perceptions_of_corruption' est : united arab emirates


In [ ]:
# Let's check if we can fill this missing value with the mean of values from other years.
# Return all the values from 'perceptions_of_corruption' columns for country 'united Arab emirates'
print(df[df['country'] == 'united arab emirates']['perceptions_of_corruption'])

19      0.385830
155     0.355610
278     0.324490
406          NaN
537     0.182000
667     0.220214
800     0.223000
928     0.250000
1059    0.247000
Name: perceptions_of_corruption, dtype: float64


In [ ]:
# Calculate the mean for all the non-missing values from 'perceptions_of_corruption' column for country 'united arab emirates'. Then, replace the missing value with the mean

# Calculate the mean for all the non-missing values from 'perceptions_of_corruption column for country 'united arab emirates'.
mean_corruption = df[df['country'] == 'united arab emirates']['perceptions_of_corruption'].dropna().mean()

# Replace the missing value with the mean
df.loc[df['country'] == 'united arab emirates', 'perceptions_of_corruption'] = df.loc[df['country'] == 'united arab emirates', 'perceptions_of_corruption'].fillna(mean_corruption)


In [ ]:
print(df[df['country'] == 'united arab emirates']['perceptions_of_corruption'])

19      0.385830
155     0.355610
278     0.324490
406     0.273518
537     0.182000
667     0.220214
800     0.223000
928     0.250000
1059    0.247000
Name: perceptions_of_corruption, dtype: float64


In [ ]:
# Count values equal to 0 in the entire DataFrame
zero_values = (df == 0).sum().sum()

print(f"Number of values equal to 0 in the dataset: {zero_values}")


Number of values equal to 0 in the dataset: 35


In [ ]:
# Let's try to understand why there are 35 values equal to 0 in our dataset.
# Is it because the real result of the study was 0, or did someone replace NaN by 0 when preparing the dataset?
# Find the location of all the values equal to 0. List the names of the columns with missing values and the value corresponding to 'country' and 'year'

# Find the location of all the values equal to 0
zero_locations = df[df == 0].stack().index.tolist()

# Extract relevant information for each zero value
zero_info = []
for index, column in zero_locations:
    country = df.loc[index, 'country']
    year = df.loc[index, 'year']
    zero_info.append({'country': country, 'year': year, 'column': column})

# Print the information for each zero value
for info in zero_info:
    print(f"Country: {info['country']}, Year: {info['year']}, Column: {info['column']}")


Country: indonesia, Year: 2015, Column: perceptions_of_corruption
Country: greece, Year: 2015, Column: generosity
Country: iraq, Year: 2015, Column: freedom_to_make_life_choices
Country: sierra leone, Year: 2015, Column: healthy_life_expectancy
Country: bosnia and herzegovina, Year: 2016, Column: perceptions_of_corruption
Country: greece, Year: 2016, Column: generosity
Country: sierra leone, Year: 2016, Column: healthy_life_expectancy
Country: togo, Year: 2016, Column: social_support
Country: greece, Year: 2017, Column: generosity
Country: bosnia and herzegovina, Year: 2017, Column: perceptions_of_corruption
Country: moldova, Year: 2018, Column: perceptions_of_corruption
Country: greece, Year: 2018, Column: generosity
Country: bosnia and herzegovina, Year: 2018, Column: perceptions_of_corruption
Country: sierra leone, Year: 2018, Column: healthy_life_expectancy
Country: moldova, Year: 2019, Column: perceptions_of_corruption
Country: greece, Year: 2019, Column: generosity
Country: afgha

In [ ]:
# Quelles sont les valeurs 'generosity' pour 'greece' pour les 'year' '2021', '2022' et '2023' ?
print(df[(df['country'] == 'greece') & (df['year'].isin([2021, 2022, 2023]))]['generosity'])
# Quelle est la moyenne pour tous les pays ?
print(df['generosity'].mean())

842     0.000
961     0.015
1091    0.008
Name: generosity, dtype: float64
0.19498117319604474


**Conclusion** : Étant donné que les valeurs pour 2022 et 2023 sont très faibles, et que la moyenne pour tous les pays est environ 10 fois plus élevée, nous pouvons considérer qu'il s'agit du véritable résultat de l'étude. Il est probable que la fragilité économique qu'a connue la Grèce ait eu une corrélation négative avec la générosité. Cette tendance pourrait indiquer que les difficultés économiques ont réduit la capacité des citoyens à donner ou à participer à des actes de générosité, affectant ainsi les résultats observés pour ces années.

In [ ]:
# Quelles sont les valeurs 'healthy_life_expectancy' pour 'sierra leone' ? (pas la moyenne). # Quelle est la moyenne pour tous les pays ?

# Quelles sont les valeurs 'healthy_life_expectancy' pour 'sierra leone' ? (pas la moyenne).
print(df[df['country'] == 'sierra leone']['healthy_life_expectancy'])
# Quelle est la moyenne pour tous les pays ?
print(df['healthy_life_expectancy'].mean())


102     0.000000
223     0.000000
351     0.005565
487     0.000000
629     0.242000
768     0.203954
898     0.100000
1029    0.273000
1160    0.092000
Name: healthy_life_expectancy, dtype: float64
0.6126270998873603



**Conclusion** : Il est difficile de comprendre les fortes variations d'une année à l'autre. Néanmoins, l'existence de valeurs aussi faibles que 0,005565 et 0,092000 semble indiquer que la valeur 0 est probable dans le contexte de la Sierra Leone.

Il est également important de rappeler que la Sierra Leone a été déclarée exempte de la maladie à virus Ebola le 17 mars 2016, après une épidémie dévastatrice qui a ravagé l'Afrique de l'Ouest entre 2013 et 2015. Néanmoins, d'autres maladies, comme le choléra, le typhus, la malaria, ainsi que la fièvre de Lassa, restent fréquentes, avec des cas récents signalés dans le nord du pays.

**Conclusion concernant les valeurs égales à 0** : Il ne s'agit pas de valeurs manquantes remplacées par 0, mais bien des véritables résultats de l'étude.


## Modeling

### Preprocessing

In [ ]:
# Select relevant columns for X and y
X = df[['gdp_per_capita', 'social_support', 'healthy_life_expectancy',
         'freedom_to_make_life_choices', 'generosity', 'perceptions_of_corruption']]
y = df['happiness_score']

In [ ]:
# Split into train and test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [ ]:
# Scale
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

### Linear Regression

In [ ]:
# Train a Linear Regression model that predicts happiness_score (our y) based on our X variables.

from sklearn.linear_model import LinearRegression
model_1 = LinearRegression().fit(X_train_scaled, y_train)

### Evaluation

Goals:
*   What is the R2 score on the train data?
*   What is the MSE (mean squared error)?
*   What about the MAE (mean absolute error)?

In [ ]:
# Score the model

from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

# Make predictions on the test set
y_pred = model_1.predict(X_test_scaled)

# Evaluate the model
r2 = r2_score(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print("R-squared:", r2)
print("Mean Squared Error:", mse)
print("Mean Absolute Error:", mae)

R-squared: 0.7021796560043735
Mean Squared Error: 0.32135255383898814
Mean Absolute Error: 0.4210575006854483


Maintenant, essayons d'améliorer notre modèle.

## Data cleaning



### Valeurs manquantes

In [ ]:
# prompt: afficher toutes les valeurs gdp_per_capita pour country "Venezuela"

print(df[df['country'] == 'venezuela']['gdp_per_capita'])


21      1.044240
167     1.133670
331     1.128431
478     0.996000
614     0.960000
736     0.770239
872     0.852000
1002    0.000000
1120    0.000000
Name: gdp_per_capita, dtype: float64
